# Question 1

## Scrape the data from the Wikipedia page into the pandas dataframe

### Importing all the required libraries

In [263]:
import requests
from bs4 import BeautifulSoup as bs

import pandas as pd
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim
import folium
from IPython.core.display import HTML

In [232]:
wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### Using BeautifulSoup scrape the wikipedia page for the table data and store it into the list

In [233]:
response = requests.get(wikipedia_url)
soup = bs(response.text, 'lxml')

table = soup.find('table', attrs={'class':'wikitable'})
table_rows = table.find_all('tr')

output = []
for tr in table_rows[1:]:
    td = tr.find_all('td')
    row = [tr.text.strip('\n') for tr in td]
    output.append(row)

### Convert list into the pandas dataframe

In [234]:
df = pd.DataFrame(output, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [235]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Only processing the cells that have an assigned borough. Ignoring the cells with a borough that is Not assigned. Droping row where borough is "Not assigned"

In [236]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Question 2

## Use the csv file to create dataframe with longitude and latitude values

### We will be using a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [237]:
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Mergin both dataframes

In [238]:
output_df = pd.merge(df, geospatial_data, left_on='PostalCode', right_on='Postal Code')
output_df.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


In [239]:
output_df.columns

Index(['PostalCode', 'Borough', 'Neighborhood', 'Postal Code', 'Latitude',
       'Longitude'],
      dtype='object')

### Dropping repetitive column 

In [240]:
output_df.drop('Postal Code', axis=1, inplace=True)

In [241]:
output_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Question 3

## Explore and cluster the neighborhoods in Toronto

### Use geopy library to get the latitude and longitude values of Toronto, Canada.

In [242]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top.

In [264]:
canada_map = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

for lat, lng, label in zip(output_df['Latitude'], output_df['Longitude'], output_df['Neighborhood']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(canada_map)

HTML(canada_map._repr_html_())

### Define Foursquare Credentials and Version
#### (NOTE: THESE CREDENTIALS WON'T BE VISIBLE IN GITHUB)

In [244]:
CLIENT_ID = 'XXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXX' # your Foursquare Secret

### Create a function to get the all the venues nearby to all the neighborhoods in Toronto

In [245]:
def getNearbyVenues(names, latitudes, longitudes, VERSION='20180605', radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood.

In [246]:
canada_venues = getNearbyVenues(names=output_df['Neighborhood'],latitudes=output_df['Latitude'],longitudes=output_df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

### Let's check how many venues were returned for each neighborhood

In [247]:
canada_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,57,57,57,57,57,57
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,18,18,18,18,18,18


### Let's find out how many unique categories can be curated from all the returned venues

In [248]:
print('There are {} uniques categories.'.format(len(canada_venues['Venue Category'].unique())))

There are 266 uniques categories.


### Analyze Each Neighborhood

In [249]:
# one hot encoding
canada_venues_one_hot = pd.get_dummies(canada_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
canada_venues_one_hot['Neighborhood'] = canada_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [canada_venues_one_hot.columns[-1]] + list(canada_venues_one_hot.columns[:-1])

canada_venues_one_hot = canada_venues_one_hot[fixed_columns]

canada_venues_one_hot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [250]:
canada_venues_grouped = canada_venues_one_hot.groupby('Neighborhood').mean().reset_index()
canada_venues_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
1,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.052632,0.0000,0.000000,0.000000,0.0,0.000000
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
5,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.017544,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
8,Business reply mail Processing Centre,0.055556,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.058824,...,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000


### Let's print each neighborhood along with the top 5 most common venues

In [251]:
num_top_venues = 5

for hood in canada_venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = canada_venues_grouped[canada_venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2             Clothing Store  0.25
3  Latin American Restaurant  0.25
4  Middle Eastern Restaurant  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.18
1                 Gym  0.09
2  Athletics & Sports  0.09
3        Dance Studio  0.09
4         Coffee Shop  0.09


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0            Bank  0.11
1     Coffee Shop  0.11
2        Pharmacy  0.05
3  Ice Cream Shop  0.05
4     Bridal Shop  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.09
2      Sandwich Place  0.09
3          Re

4   Seafood Restaurant  0.08


----North Park, Maple Leaf Park, Upwood Park----
                        venue  freq
0            Basketball Court  0.25
1                        Park  0.25
2                      Bakery  0.25
3  Construction & Landscaping  0.25
4                 Yoga Studio  0.00


----North Toronto West----
                  venue  freq
0        Clothing Store  0.15
1           Coffee Shop  0.10
2           Yoga Studio  0.05
3   Sporting Goods Shop  0.05
4  Fast Food Restaurant  0.05


----Northwest----
                       venue  freq
0        Rental Car Location   0.5
1                  Drugstore   0.5
2                Yoga Studio   0.0
3  Middle Eastern Restaurant   0.0
4        Monument / Landmark   0.0


----Northwood Park, York University----
                  venue  freq
0        Massage Studio  0.17
1           Coffee Shop  0.17
2  Caribbean Restaurant  0.17
3                   Bar  0.17
4    Miscellaneous Shop  0.17


----Old Mill South, King's Mill Park, Sun

### Create a function to sort the venues in descending order.

In [252]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create the new dataframe and display the top 10 venues for each neighborhood.

In [253]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = canada_venues_grouped['Neighborhood']

for ind in np.arange(canada_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(canada_venues_grouped.iloc[ind, :], num_top_venues)


In [254]:
canada_venues_grouped_clustering = canada_venues_grouped.drop('Neighborhood', 1)
canada_venues_grouped_clustering.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


### Cluster Neighborhoods into 5 clusters.

In [255]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(canada_venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [256]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

canada_merged = output_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
canada_merged = canada_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
canada_merged.dropna(subset=['Cluster Labels'], inplace=True)
canada_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Fast Food Restaurant,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,French Restaurant,Portuguese Restaurant,Coffee Shop,Pizza Place,Hockey Arena,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Restaurant,Yoga Studio,Hotel
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Women's Store,Furniture / Home Store,Clothing Store,Coffee Shop,Boutique,Miscellaneous Shop,Athletics & Sports,Event Space,Accessories Store,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Distribution Center,Restaurant,Park,Mexican Restaurant,Juice Bar,Japanese Restaurant,Italian Restaurant,Hobby Shop


### Visualize the resulting clusters

In [265]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(canada_merged['Latitude'], canada_merged['Longitude'], canada_merged['Neighborhood'], canada_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
HTML(map_clusters._repr_html_())

### Examine all the Clusters

#### Cluster 1

In [258]:
canada_merged.loc[canada_merged['Cluster Labels'] == 0, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,York,0.0,Park,Field,Hockey Arena,Trail,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dance Studio
21,York,0.0,Park,Pool,Women's Store,College Rec Center,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
35,East York,0.0,Park,Convenience Store,Coffee Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
45,North York,0.0,Park,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store,Dance Studio
61,Central Toronto,0.0,Park,Bus Line,Swim School,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Ethiopian Restaurant
64,York,0.0,Park,Convenience Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Women's Store,Deli / Bodega
66,North York,0.0,Park,Convenience Store,Bank,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Deli / Bodega
85,Scarborough,0.0,Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
91,Downtown Toronto,0.0,Park,Playground,Trail,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Eastern European Restaurant,Distribution Center,Cupcake Shop
98,Etobicoke,0.0,Park,River,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Ethiopian Restaurant,Discount Store


#### Cluster 2

In [259]:
canada_merged.loc[canada_merged['Cluster Labels'] == 1, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,French Restaurant,Portuguese Restaurant,Coffee Shop,Pizza Place,Hockey Arena,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,Downtown Toronto,1.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Restaurant,Yoga Studio,Hotel
3,North York,1.0,Women's Store,Furniture / Home Store,Clothing Store,Coffee Shop,Boutique,Miscellaneous Shop,Athletics & Sports,Event Space,Accessories Store,Vietnamese Restaurant
4,Downtown Toronto,1.0,Coffee Shop,Sushi Restaurant,Distribution Center,Restaurant,Park,Mexican Restaurant,Juice Bar,Japanese Restaurant,Italian Restaurant,Hobby Shop
7,North York,1.0,Gym,Restaurant,Coffee Shop,Asian Restaurant,Japanese Restaurant,Beer Store,Sporting Goods Shop,Italian Restaurant,Shopping Mall,Sandwich Place
8,East York,1.0,Pizza Place,Gym / Fitness Center,Breakfast Spot,Fast Food Restaurant,Pharmacy,Bank,Gastropub,Athletics & Sports,Intersection,Women's Store
9,Downtown Toronto,1.0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Hotel
10,North York,1.0,Japanese Restaurant,Sushi Restaurant,Pizza Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Women's Store
11,Etobicoke,1.0,Golf Course,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
12,Scarborough,1.0,History Museum,Bar,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Deli / Bodega


#### Cluster 3

In [260]:
canada_merged.loc[canada_merged['Cluster Labels'] == 2, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Park,Fast Food Restaurant,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
6,Scarborough,2.0,Fast Food Restaurant,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Cluster 4

In [261]:
canada_merged.loc[canada_merged['Cluster Labels'] == 3, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,North York,3.0,Park,Bakery,Basketball Court,Construction & Landscaping,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
101,Etobicoke,3.0,Construction & Landscaping,Baseball Field,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store


#### Cluster 5

In [262]:
canada_merged.loc[canada_merged['Cluster Labels'] == 4, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4.0,Baseball Field,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
